In [1]:
import turicreate as tc

ModuleNotFoundError: No module named 'turicreate'

# import data

In [3]:
product = tc.SFrame("amazon_baby.sframe")

In [4]:
product.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


# build word count vector

In [8]:
product["word_count"] = tc.text_analytics.count_words(product['review'])

In [9]:
product.head(5)

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1.0,'stripping': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'moist': 1.0, 'osocozy': ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0,'comfortable': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'little': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ..."


## find the product with most feedbacks

https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.groupby.html

In [17]:
#skill : groupby
most_review = product.groupby('name', operations={'count': tc.aggregate.COUNT()}).sort('count', ascending=False)
print(most_review.head(5))
print(most_review.tail(5))
print(most_review[0])

+--------------------------------+-------+
|              name              | count |
+--------------------------------+-------+
| Vulli Sophie the Giraffe T...  |  785  |
| Simple Wishes Hands-Free B...  |  562  |
| Infant Optics DXR-5 2.4 GH...  |  561  |
| Baby Einstein Take Along Tunes |  547  |
| Cloud b Twilight Constella...  |  520  |
+--------------------------------+-------+
[5 rows x 2 columns]

+-------------------------------+-------+
|              name             | count |
+-------------------------------+-------+
| Koala Baby Wallborders - L... |   1   |
| Pam Grace Creations Portab... |   1   |
| Medela PersonalFit Breasts... |   1   |
| KerrBear Kids - Rainbow Le... |   1   |
| Classic Single Sunshade Ca... |   1   |
+-------------------------------+-------+
[5 rows x 2 columns]

{'name': 'Vulli Sophie the Giraffe Teether', 'count': 785}


# build a dataset for 'Vulli Sophie the Giraffe Teether'

In [19]:
giraffe_reviews = product[product["name"] == 'Vulli Sophie the Giraffe Teether']
print(len(giraffe_reviews))

785


In [20]:
giraffe_reviews.show()

Materializing SFrame

Warning: Skipping column 'word_count'. Unable to show columns of type 'dictionary'; only [int, float, str] can be shown.

Further warnings of unsupported type will be suppressed.

# add a new column called sentiment as an evaluation of ML

In [24]:
# ingore the review with 3-star rating
product = product[product['rating'] != 3]

# positive sentiment = 4-star or 5-star reviews
product['sentiment'] = product['rating'] >= 4  # add new columns
# note : if rating is 4 or 5, sentiment is 1, whereas sentiment is 0 when rating is 1 or 2

In [31]:
product = product.sort("rating" ,ascending=False)

In [32]:
print(product.head(5))
print('')
print(product.tail(5))

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|           word_count          | sentiment |
+-------------------------------+-----------+
| {'recommend': 1.0, 'moist'... |     1     |
| {'quilt': 1.0, 'comfortabl... |     1     |
| {'tool': 1.0, 'clever': 1.... |     1     |
| {'rock': 1.0, 'many': 1.0,... |     1     |
| {

# build a setiment classifier

logistic_classifier

https://apple.github.io/turicreate/docs/api/generated/turicreate.logistic_classifier.LogisticClassifier.predict.html?highlight=predict%20logistic#turicreate.logistic_classifier.LogisticClassifier.predict

In [33]:
train_data, test_data = product.random_split(0.8, seed = 0)

In [55]:
sentiment_model = tc.logistic_classifier.create(train_data, target='sentiment', features=['word_count'],
                                                validation_set = test_data )

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.672492  | 2.040353     | 0.938800          | 0.913794            |

| 3         | 12       | 2.672492  | 2.345188     | 0.927770          | 0.900613            |

| 4         | 14       | 1.336765  | 2.803110     | 0.946076          | 0.916617            |

| 9         | 20       | 1.336765  | 4.619806     | 0.978051          | 0.918628            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [37]:
sentiment_model.evaluate(train_data)

{'accuracy': 0.9780513758167976,
 'auc': 0.9945356620637502,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+--------+
 | target_label | predicted_label | count  |
 +--------------+-----------------+--------+
 |      1       |        0        |  1118  |
 |      1       |        1        | 111136 |
 |      0       |        1        |  1811  |
 |      0       |        0        | 19383  |
 +--------------+-----------------+--------+
 [4 rows x 3 columns],
 'f1_score': 0.9869938410575442,
 'log_loss': 0.06354707285271588,
 'precision': 0.9839659309233534,
 'recall': 0.9900404439930871,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+--------+-------+
 | threshold |        fpr         |        tpr         |   p    |   n   |
 +-----------+--------------------+--------------------+--------+--

In [56]:
product['predicted_result'] = sentiment_model.predict(product, output_type='probability')

In [57]:
product[:3]

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	predicted_result	float

Rows: 3

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+--------------------+
|           word_count          | sentiment |  predicted_result  |
+-------------------------------+-----------+--------------------+
| {'recommend': 1.0, 'moist'... |     1     | 0.9997951050924122 |
| {'quilt': 1.0, 'comfortabl... |     1     | 0.9982055197915087 |
| {'tool': 1.0, 'clever': 1.... |     1     | 0.9999982995845093 |
+-------------------------------+-----------+--------------------+
[3 rows x 6 columns]

In [58]:
product[-5:-1]
# we csan know that the predicted_result with lower prbability means that the setiment is 

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	predicted_result	float

Rows: 4

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Manito Elegance Plus Strol... | I tried it on my umbrella ... |  1.0   |
| GroVia BioLiners One Size ... | These liners caused my clo... |  1.0   |
| Summer Infant Multi-Use De... | Never have I had such a pr... |  1.0   |
| Summer Infant Multi-Use De... | This is the poorest qualit... |  1.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+------------------------+
|           word_count          | sentiment |    predicted_result    |
+-------------------------------+-----------+------------------------+
| {'50': 1.0, 'film': 1.0, '... |     0     | 5.1427062611874196e-08 |
| {'design': 1.0, 'terrible'... |     0     |  0.044403419062150784  |
| {'baby': 1.0, 'for': 1.0, ... |     0     | 5.124009078604721e-05  |
| {'item': 1.0, 'another': 2... |     0     |  4.29905422657072e-06  |
+-------------------------------+-----------+------------------------+
[4 rows x 6 columns]

# add this model to understand better the giraffe_review

In [59]:
giraffe_with_prediction_result = product[product['name'] == 'Vulli Sophie the Giraffe Teether' ]

In [60]:
giraffe_with_prediction_result.sort('predicted_result', ascending=False)

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'09': 1.0, '11': 1.0,'recommended': 1.0, ' ...",1
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'use': 1.0, 'get': 1.0,'will': 1.0, 'where': ...",1
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'explore': 1.0,'adventures': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5.0,"{'future': 1.0, 'use':1.0, 'am': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,My 8 week old LOVESSophie. The rubber feels ...,5.0,"{'highly': 1.0, 'now':1.0, 'i': 2.0, 'hit': ...",1
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4.0,"{'teether': 1.0, 'want':1.0, 'price': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'what': 1.0, 'are': 1.0,'have': 1.0, 'worth': ...",1
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'parents': 1.0,'recommend': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'well': 1.0, 'natural':1.0, 'makers': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5.0,"{'penny': 1.0, 'worth':1.0, 'definitely': 1.0, ...",1
